## Preloaded

In [0]:
!pip3 install networkit

     |████████████████████████████████| 1.2MB 2.7MB/s 
  Created wheel for networkit: filename=networkit-5.0.1-cp36-cp36m-linux_x86_64.whl size=7120732 sha256=f22384861edb71d1f8fe2583223f7fb277022a58df035a94333cce0bb5fc2f6d
  Stored in directory: /root/.cache/pip/wheels/8c/c7/c8/5e7247f1ff3539c7517dff48774172cd9d202ba4452eed0bd9
Successfully built networkit


In [0]:
from collections import namedtuple
from functools import wraps
from math import ceil, floor
from random import sample, randint
from scipy.stats import kendalltau
import networkx as nx
import networkit as nk
import numpy as np
import os
from functools import partial
import matplotlib.pyplot as plt
%matplotlib inline
from networkit import *
import pandas as pd

In [0]:
def nk_wrapper(nk_centrality, *args):
  def f(g):
    cm_values = nk_centrality(nk.nxadapter.nx2nk(g), *args).run().scores()
    return dict(zip(g.nodes, cm_values))
  f.__name__=nk_centrality.__name__
  return f

## Jaccard

In [0]:
#defintion jaccard sim

def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [0]:
def nk_wrapper(nk_centrality, *args):
  def f(g):
    cm_values = nk_centrality(nk.nxadapter.nx2nk(g), *args).run().scores()
    return dict(zip(g.nodes, cm_values))
  f.__name__=nk_centrality.__name__
  return f

In [0]:
# error formula

def remove_edges_uniform(graph, alpha):
    edges_to_remove = sample(graph.edges, ceil(graph.number_of_edges() * alpha))
    new_g = graph.copy()

    new_g.remove_edges_from(edges_to_remove)
    return new_g


def remove_nodes_uniform(graph, alpha):
    nodes_to_keep = sample(graph.nodes, floor(graph.number_of_nodes() * (1 - alpha)))

    return graph.subgraph(nodes_to_keep)


def remove_edges_proportional_degree(mygraph, alpha):
    new_g = mygraph.copy()

    deg = nx.degree(mygraph)
    degsum = np.array([deg[x] + deg[y] for x, y in mygraph.edges()])

    np.random.seed(int.from_bytes(os.urandom(4), byteorder='big'))
    edge_ids = np.random.choice(mygraph.number_of_edges(), size=ceil(mygraph.number_of_edges() * alpha),
                                replace=False, p=degsum / degsum.sum())

    edges_to_remove = np.array(mygraph.edges)[edge_ids, :]

    new_g.remove_edges_from(edges_to_remove)
    return new_g


def add_edges_random(graph, alpha):
    edges_to_add = ceil(graph.number_of_edges() * alpha)
    new_G = graph.copy()
    N = new_G.number_of_nodes()
    edgelist = set([tuple(sorted(e)) for e in new_G.edges()])

    cnt = 0
    new_edges = set()

    while cnt < edges_to_add:
        new_edge = tuple(sorted([randint(0, N - 1), randint(0, N - 1)]))
        if new_edge[0] == new_edge[1]:
            continue
        elif not (new_edge in edgelist or new_edge in new_edges):
            new_edges.add(new_edge)
            cnt += 1

    new_G.add_edges_from(new_edges)
    return new_G

## 2. Load in the network

In [0]:
#read network in

#with open('/content/drive/My Drive/out.petster-friendships-hamster-uniq') as f:
  print(f.readlines()[:5])

In [0]:
#read network in

with open("out.petster-friendships-hamster-uniq") as f:
  print(f.readlines()[:5])

['% sym unweighted\n', '99 98\n', '999 550\n', '999 42\n', '999 25\n']


In [0]:
#interaction_graph = nx.read_edgelist("/out.petster-friendships-hamster-uniq", create_using = nx.Graph(), comments = "%")
interaction_graph = nx.read_edgelist("out.petster-friendships-hamster-uniq", create_using = nx.Graph(), comments = "%")

In [0]:
connected_subgraph=max(nx.connected_component_subgraphs(interaction_graph), key=len)


In [0]:
nx.info(connected_subgraph)

'Name: \nType: Graph\nNumber of nodes: 1788\nNumber of edges: 12476\nAverage degree:  13.9553'

## 3. Input

In [0]:
#define input
hidden_network=connected_subgraph
alpha1=0.1
alpha2=0.3
error_measure_aer=partial(add_edges_random, alpha=alpha2) 
error_measure_reu=partial(remove_edges_uniform, alpha=alpha2)
error_measure_repd=partial(remove_edges_proportional_degree, alpha=alpha2)
error_measure_rnu=partial(remove_nodes_uniform, alpha=alpha2)
centralityd=nk_wrapper(nk.centrality.DegreeCentrality, True)
centralityb=nk_wrapper(nk.centrality.Betweenness)
centralityc=nk_wrapper(nk.centrality.Closeness, True, True)
#centralitye=nx.eigenvector_centrality
centralitye=nk_wrapper(nk.centrality.EigenvectorCentrality)
centralityp=nk_wrapper(nk.centrality.PageRank)


In [0]:
def robustness1(hidden_network,centrality,error_measure, iterations=100):
    jaccard_values = []
    for _ in range(iterations):
        network_error_measure=error_measure(hidden_network)
        network_error_measure_again = error_measure(network_error_measure)
        network_measured_error_centrality =centrality(network_error_measure_again)
        network_hidden_error_centrality = centrality(network_error_measure)
        network_measured_top10 = np.array(sorted(network_measured_error_centrality.items(), key=lambda x: x[1], 
                                                reverse=True)[:int(len(network_measured_error_centrality)*0.1)])
        network_hidden_top10 = np.array(sorted(network_hidden_error_centrality.items(), key=lambda x: x[1], 
                                              reverse=True)[:int(len(network_hidden_error_centrality)*0.1)])

        jaccard_result = jaccard_similarity(network_measured_top10[:,0],network_hidden_top10[:,0])

        jaccard_values.append(jaccard_result) 

    return namedtuple("robustness_actual", "mean, sd")(np.mean(jaccard_values),
                                                                np.std(jaccard_values))
def robustness2(hidden_network,centrality,error_measure, iterations=100):
    jaccard_values = []
    for _ in range(iterations):
        network_error_measure=error_measure(hidden_network)
        network_error_measure_again = error_measure(network_error_measure)
        network_measured_error_centrality =centrality(network_error_measure_again)
        network_hidden_error_centrality = centrality(network_error_measure)
        network_measured_top10 = np.array(sorted(network_measured_error_centrality.items(), key=lambda x: x[1], 
                                                reverse=True)[:int(len(network_measured_error_centrality)*0.1)])
        network_hidden_top10 = np.array(sorted(network_hidden_error_centrality.items(), key=lambda x: x[1], 
                                              reverse=True)[:int(len(network_hidden_error_centrality)*0.1)])

        jaccard_result = jaccard_similarity(network_measured_top10[:,0],network_hidden_top10[:,0])

        jaccard_values.append(jaccard_result) 

    return namedtuple("robustness_estimate", "mean, sd")(np.mean(jaccard_values),
                                                                np.std(jaccard_values))

## 4. Code

## add_edges_random 10 100

In [0]:
robustness1(hidden_network,centralityd,error_measure_aer,)

robustness_actual(mean=0.7497318430324621, sd=0.003416791244088905)

In [0]:
robustness2(error_measure_aer(hidden_network),centralityd,error_measure_aer,)

robustness_estimate(mean=0.8193505183298327, sd=0.0032067348396902277)

In [0]:
robustness1(hidden_network,centralityb,error_measure_aer,)

robustness_actual(mean=0.3725959853617982, sd=0.022179552875438716)

In [0]:
robustness2(error_measure_aer(hidden_network),centralityb,error_measure_aer,)

robustness_estimate(mean=0.5221280938280244, sd=0.018426792800566156)

In [0]:
robustness1(hidden_network,centralityc,error_measure_aer,)

robustness_actual(mean=0.7502414873854032, sd=0.0031782707587926524)

In [0]:
robustness2(error_measure_aer(hidden_network),centralityc,error_measure_aer,)

robustness_estimate(mean=0.8200583062940798, sd=0.003110124888607971)

In [0]:
robustness1(hidden_network,centralitye,error_measure_aer,)

robustness_actual(mean=0.7500582133463211, sd=0.003388232418500961)

In [0]:
robustness2(error_measure_aer(hidden_network),centralitye,error_measure_aer,)

robustness_estimate(mean=0.8194078878524843, sd=0.002783628352568849)

In [0]:
robustness1(hidden_network,centralityp,error_measure_aer,)

robustness_actual(mean=0.645895659707001, sd=0.014084296838966394)

In [0]:
robustness2(error_measure_aer(hidden_network),centralityp,error_measure_aer,)

robustness_estimate(mean=0.6270721415907535, sd=0.015933967293243815)

## remove_edges_uniform 10 100

In [0]:
robustness1(hidden_network,centralityd,error_measure_reu,)

robustness_actual(mean=0.9337481467093229, sd=0.01530085042765771)

In [0]:
robustness2(error_measure_reu(hidden_network),centralityd,error_measure_reu,)

robustness_estimate(mean=0.9363887979223776, sd=0.0160480122738406)

In [0]:
robustness1(hidden_network,centralityb,error_measure_reu,)

robustness_actual(mean=0.8776629874483324, sd=0.02460862043025914)

In [0]:
robustness2(error_measure_reu(hidden_network1),centralityb,error_measure_reu,)

robustness_estimate(mean=0.8813221852186309, sd=0.02416026465493863)

In [0]:
robustness1(hidden_network,centralityc,error_measure_reu,)

robustness_actual(mean=0.911227363087431, sd=0.020897203829298498)

In [0]:
robustness2(error_measure_reu(hidden_network),centralityc,error_measure_reu,)

robustness_estimate(mean=0.8991976495730318, sd=0.02075795916577129)

In [0]:
robustness1(hidden_network,centralitye,error_measure_reu,)

robustness_actual(mean=0.9050043534028468, sd=0.021435420938643777)

In [0]:
robustness2(error_measure_reu(hidden_network),centralitye,error_measure_reu,)

robustness_estimate(mean=0.9007364680169981, sd=0.021575909558872886)

In [0]:
robustness1(hidden_network,centralityp,error_measure_reu,)

robustness_actual(mean=0.9359001763422037, sd=0.018186251919698238)

In [0]:
robustness2(error_measure_reu(hidden_network),centralityp,error_measure_reu,)

robustness_estimate(mean=0.9363952669547524, sd=0.016409965296525966)

## remove_edges_proportional_degree 10 100


In [0]:
robustness1(hidden_network,centralityd,error_measure_repd,)

robustness_actual(mean=0.9290921559706549, sd=0.018627254129910814)

In [0]:
robustness2(error_measure_repd(hidden_network1),centralityd,error_measure_repd,)

robustness_estimate(mean=0.9308379934270063, sd=0.016887392545819283)

In [0]:
robustness1(hidden_network,centralityb,error_measure_repd,)

robustness_actual(mean=0.9084311290074034, sd=0.019385065172385037)

In [0]:
robustness2(error_measure_repd(hidden_network),centralityb,error_measure_repd,)

robustness_estimate(mean=0.9031677281400532, sd=0.021242090124685434)

In [0]:
robustness1(hidden_network,centralityc,error_measure_repd,)

robustness_actual(mean=0.8711844551401012, sd=0.02199021155164268)

In [0]:
robustness2(error_measure_repd(hidden_network),centralityc,error_measure_repd,)

robustness_estimate(mean=0.8686000967316084, sd=0.020558741187569656)

In [0]:
robustness1(hidden_network,centralitye,error_measure_repd,)

robustness_actual(mean=0.8807751114766095, sd=0.022127303971765603)

In [0]:
robustness2(error_measure_repd(hidden_network),centralitye,error_measure_repd,)

robustness_estimate(mean=0.889111153285541, sd=0.02021742802385381)

In [0]:
robustness1(hidden_network,centralityp,error_measure_repd,)

robustness_actual(mean=0.9351182350328768, sd=0.015544430228471354)

In [0]:
robustness2(error_measure_repd(hidden_network),centralityp,error_measure_repd,)

robustness_estimate(mean=0.9335699447060928, sd=0.017205630751079736)

## remove_nodes_uniform 10 100

In [0]:
robustness1(hidden_network,centralityd,error_measure_rnu,)

robustness_actual(mean=0.8415435727750432, sd=0.028970387952713048)

In [0]:
robustness2(error_measure_rnu(hidden_network),centralityd,error_measure_rnu,)

robustness_estimate(mean=0.8465378410393753, sd=0.03149385029607774)

In [0]:
robustness1(hidden_network,centralityb,error_measure_rnu,)

robustness_actual(mean=0.7905847350294468, sd=0.0370081950263111)

In [0]:
robustness2(error_measure_rnu(hidden_network),centralityb,error_measure_rnu,)

robustness_estimate(mean=0.7784010462138852, sd=0.040134538101002294)

In [0]:
robustness1(hidden_network,centralityc,error_measure_rnu,)

robustness_actual(mean=0.8259461964569048, sd=0.04450905827034137)

In [0]:
robustness2(error_measure_rnu(hidden_network),centralityc,error_measure_rnu,)

robustness_estimate(mean=0.8296358363356694, sd=0.05170912003708723)

In [0]:
robustness1(hidden_network,centralitye,error_measure_rnu,)

robustness_actual(mean=0.818423808893368, sd=0.042008166031869125)

In [0]:
robustness2(error_measure_rnu(hidden_network),centralitye,error_measure_rnu,)

robustness_estimate(mean=0.8247740613708966, sd=0.042945532293218916)

In [0]:
robustness1(hidden_network,centralityp,error_measure_rnu,)

robustness_actual(mean=0.8429260013678755, sd=0.03028000655017636)

In [0]:
robustness2(error_measure_rnu(hidden_network),centralityp,error_measure_rnu,)

robustness_estimate(mean=0.8429376246436804, sd=0.02789046154023033)

In [0]:
alpha1=alpha2

## add_edges_random 30 100 runs

In [0]:
robustness1(hidden_network,centralityd,error_measure_aer,)

robustness_actual(mean=0.6795437346127838, sd=0.001497443998825027)

In [0]:
robustness2(error_measure_aer(hidden_network),centralityd,error_measure_aer,)

robustness_estimate(mean=0.7579124122846648, sd=0.003298488167226517)

In [0]:
robustness1(hidden_network,centralityb,error_measure_aer,)

robustness_actual(mean=0.3966258988429263, sd=0.012950224803130645)

In [0]:
robustness2(error_measure_aer(hidden_network),centralityb,error_measure_aer,)

robustness_estimate(mean=0.5300378139186867, sd=0.014880345954146458)

In [0]:
robustness1(hidden_network,centralityc,error_measure_aer,)

robustness_actual(mean=0.13499868587750366, sd=0.011508808268583396)

In [0]:
robustness2(error_measure_aer(hidden_network),centralityc,error_measure_aer,)

robustness_estimate(mean=0.0, sd=0.0)

In [0]:
robustness1(hidden_network,centralitye,error_measure_aer,)

robustness_actual(mean=0.6792759338410406, sd=0.0015590419497469496)

In [0]:
robustness2(error_measure_aer(hidden_network),centralitye,error_measure_aer,)

robustness_estimate(mean=0.7620006241175621, sd=0.00146064464708162)

In [0]:
robustness1(hidden_network,centralityp,error_measure_aer,)

robustness_actual(mean=0.5571845774895292, sd=0.014449690162645462)

In [0]:
robustness2(error_measure_aer(hidden_network),centralityp,error_measure_aer,)

robustness_estimate(mean=0.5651313360848829, sd=0.012002921476733234)

## remove_edges_uniform 30 100 runs

In [0]:
robustness1(hidden_network,centralityd,error_measure_reu,)

robustness_actual(mean=0.8691808797109739, sd=0.020217098396689115)

In [0]:
robustness2(error_measure_reu(hidden_network),centralityd,error_measure_reu,)

robustness_estimate(mean=0.8573267697968505, sd=0.021918531093094242)

In [0]:
robustness1(hidden_network,centralityb,error_measure_reu,)

robustness_actual(mean=0.7515643912226446, sd=0.02557338148728903)

In [0]:
robustness2(error_measure_reu(hidden_network),centralityb,error_measure_reu,)

robustness_estimate(mean=0.7434326156329156, sd=0.025883669781895515)

In [0]:
robustness1(hidden_network,centralityc,error_measure_reu,)

robustness_actual(mean=0.8022036532307056, sd=0.023741547829637975)

In [0]:
robustness2(error_measure_reu(hidden_network),centralityc,error_measure_reu,)

robustness_estimate(mean=0.7639988803072103, sd=0.026562114681623134)

In [0]:
robustness1(hidden_network,centralitye,error_measure_reu,)

robustness_actual(mean=0.8047135562522186, sd=0.02534029269627291)

In [0]:
robustness2(error_measure_reu(hidden_network),centralitye,error_measure_reu,)

robustness_estimate(mean=0.7558118378262629, sd=0.026105595361672146)

In [0]:
robustness1(hidden_network,centralityp,error_measure_reu,)

robustness_actual(mean=0.8547852666048992, sd=0.024672951703515914)

In [0]:
robustness2(error_measure_reu(hidden_network),centralityp,error_measure_reu,)

robustness_estimate(mean=0.8316295197034, sd=0.02546098397051649)

## remove_edges_proportional_degree 30 100 runs


In [0]:
robustness1(hidden_network,centralityd,error_measure_repd,)

robustness_actual(mean=0.816939437799884, sd=0.024614427350472347)

In [0]:
robustness2(error_measure_repd(hidden_network),centralityd,error_measure_repd,)

robustness_estimate(mean=0.7279584187686376, sd=0.028571253745129893)

In [0]:
robustness1(hidden_network,centralityb,error_measure_repd,)

robustness_actual(mean=0.7610721043276308, sd=0.027752683089127272)

In [0]:
robustness2(error_measure_repd(hidden_network),centralityb,error_measure_repd,)

robustness_estimate(mean=0.6800432252059423, sd=0.030810374483976995)

In [0]:
robustness1(hidden_network,centralityc,error_measure_repd,)

robustness_actual(mean=0.7244971638709594, sd=0.0249381063840745)

In [0]:
robustness2(error_measure_repd(hidden_network),centralityc,error_measure_repd,)

robustness_estimate(mean=0.6431563447324372, sd=0.03015016757872722)

In [0]:
robustness1(hidden_network,centralitye,error_measure_repd,)

robustness_actual(mean=0.7632680508089668, sd=0.028436213383737013)

In [0]:
robustness2(error_measure_repd(hidden_network),centralitye,error_measure_repd,)

robustness_estimate(mean=0.6729645067572698, sd=0.033298888266593114)

In [0]:
robustness1(hidden_network,centralityp,error_measure_repd,)

robustness_actual(mean=0.8205390560318406, sd=0.02389926592105107)

In [0]:
robustness2(error_measure_repd(hidden_network),centralityp,error_measure_repd,)

robustness_estimate(mean=0.7309067006526291, sd=0.0263167109843062)

## remove_nodes_uniform 30 100 runs


In [0]:
robustness1(hidden_network,centralityd,error_measure_rnu,)

robustness_actual(mean=0.615952643912662, sd=0.039314435357100226)

In [0]:
robustness2(error_measure_rnu(hidden_network),centralityd,error_measure_rnu,)

robustness_estimate(mean=0.5940642252377915, sd=0.05326395290224939)

In [0]:
robustness1(hidden_network,centralityb,error_measure_rnu,)

robustness_actual(mean=0.5272125706383327, sd=0.04569605359548101)

In [0]:
robustness2(error_measure_rnu(hidden_network),centralityb,error_measure_rnu,)

robustness_estimate(mean=0.5289733384896028, sd=0.04714932844237512)

In [0]:
robustness1(hidden_network,centralityc,error_measure_rnu,)

robustness_actual(mean=0.5803873979550118, sd=0.05010669190559617)

In [0]:
robustness2(error_measure_rnu(hidden_network),centralityc,error_measure_rnu,)

robustness_estimate(mean=0.5403090851463904, sd=0.07205834053968735)

In [0]:
robustness1(hidden_network,centralitye,error_measure_rnu,)

robustness_actual(mean=0.5795306828731747, sd=0.059752038237122)

In [0]:
robustness2(error_measure_rnu(hidden_network),centralitye,error_measure_rnu,)

robustness_estimate(mean=0.5747981273631706, sd=0.060379079378909496)

In [0]:
robustness1(hidden_network,centralityp,error_measure_rnu,)

robustness_actual(mean=0.6115200386738912, sd=0.041186669928804806)

In [0]:
robustness2(error_measure_rnu(hidden_network),centralityp,error_measure_rnu,)

robustness_estimate(mean=0.590177436008832, sd=0.0428974036570475)

## add_edges_random 10 500

In [0]:
robustness1(hidden_network,centralityd,error_measure_aer,)

robustness_actual(mean=0.749817239664887, sd=0.0034580319728431197)

In [0]:
robustness2(error_measure_aer(hidden_network),centralityd,error_measure_aer,)

robustness_estimate(mean=0.8194291826680484, sd=0.0032130892499785484)

In [0]:
robustness1(hidden_network,centralityb,error_measure_aer,)

robustness_actual(mean=0.37272291109272954, sd=0.023110824096609056)

In [0]:
robustness2(error_measure_aer(hidden_network),centralityb,error_measure_aer,)

robustness_estimate(mean=0.5187563458883784, sd=0.019331592489869386)

In [0]:
robustness1(hidden_network,centralityc,error_measure_aer,)

robustness_actual(mean=0.7498275965162936, sd=0.003466615683524725)

In [0]:
robustness2(error_measure_aer(hidden_network),centralityc,error_measure_aer,)

robustness_estimate(mean=0.819860699740612, sd=0.0031972672167505036)

In [0]:
robustness1(hidden_network,centralitye,error_measure_aer,)

robustness_actual(mean=0.7499625583792108, sd=0.003218745484835341)

In [0]:
robustness2(error_measure_aer(hidden_network),centralitye,error_measure_aer,)

robustness_estimate(mean=0.8196580099823291, sd=0.003268631313405282)

In [0]:
robustness1(hidden_network,centralityp,error_measure_aer,)

robustness_actual(mean=0.645353294027122, sd=0.014837208230229123)

In [0]:
robustness2(error_measure_aer(hidden_network),centralityp,error_measure_aer,)

robustness_estimate(mean=0.6300432476829563, sd=0.016287142539846443)

## Keldaltau


## 1. Formula

In [0]:
def remove_edges_uniform(graph, alpha):
    edges_to_remove = sample(graph.edges, ceil(graph.number_of_edges() * alpha))
    new_g = graph.copy()

    new_g.remove_edges_from(edges_to_remove)
    return new_g


def remove_nodes_uniform(graph, alpha):
    nodes_to_keep = sample(graph.nodes, floor(graph.number_of_nodes() * (1 - alpha)))

    return graph.subgraph(nodes_to_keep)


def remove_edges_proportional_degree(mygraph, alpha):
    new_g = mygraph.copy()

    deg = nx.degree(mygraph)
    degsum = np.array([deg[x] + deg[y] for x, y in mygraph.edges()])

    np.random.seed(int.from_bytes(os.urandom(4), byteorder='big'))
    edge_ids = np.random.choice(mygraph.number_of_edges(), size=ceil(mygraph.number_of_edges() * alpha),
                                replace=False, p=degsum / degsum.sum())

    edges_to_remove = np.array(mygraph.edges)[edge_ids, :]

    new_g.remove_edges_from(edges_to_remove)
    return new_g


def add_edges_random(graph, alpha):
    edges_to_add = ceil(graph.number_of_edges() * alpha)
    new_G = graph.copy()
    N = new_G.number_of_nodes()
    edgelist = set([tuple(sorted(e)) for e in new_G.edges()])

    cnt = 0
    new_edges = set()

    while cnt < edges_to_add:
        new_edge = tuple(sorted([randint(0, N - 1), randint(0, N - 1)]))
        if new_edge[0] == new_edge[1]:
            continue
        elif not (new_edge in edgelist or new_edge in new_edges):
            new_edges.add(new_edge)
            cnt += 1

    new_G.add_edges_from(new_edges)
    return new_G


def compare_centrality_dicts_correlation(d1, d2, scipy_correlation=kendalltau):
    if set(d1) != set(d2):
        nodes = sorted(set(d1).intersection(set(d2)))
    else:
        nodes = sorted(d1)

    v1 = np.round([d1[x] for x in nodes], 12)
    v2 = np.round([d2[x] for x in nodes], 12)

    return scipy_correlation(v1, v2).correlation


def robustness_calculator_builder(centrality_measure, comparison_function=compare_centrality_dicts_correlation):
    @wraps(centrality_measure)
    def f(g0, g1):
        return compare_centrality_dicts_correlation(centrality_measure(g0), centrality_measure(g1))
    return f


def estimate_robustness(measured_network, error_mechanism, robustness_calculator, iterations=500, return_values=False):
    measured_robustness = np.array([robustness_calculator(measured_network, error_mechanism(measured_network))
                                    for _ in range(iterations)])
    vals = measured_robustness if return_values else None
    return namedtuple("robustness_estimate", "mean, sd values")(measured_robustness.mean(),
                                                                measured_robustness.std(), vals)

def true_robustness(hidden_network, error_mechanism, robustness_calculator, iterations=500, return_values=False):
    measured_robustness=np.array([robustness_calculator(hidden_network,error_mechanism(hidden_network))
                                  for _ in range(iterations)])
    vals = measured_robustness if return_values else None
    return namedtuple("robustness_true", "mean, sd values")(measured_robustness.mean(),
                                                                measured_robustness.std(), vals) 

## 2. Load in the network

In [0]:
#read network in version 1

#with open('/content/drive/My Drive/out.petster-friendships-hamster-uniq') as f:
  print(f.readlines()[:5])

In [0]:
#read network in version 2 if you have it in the table of content uploaded

with open("out.petster-friendships-hamster-uniq") as f:
  print(f.readlines()[:5])

['% sym unweighted\n', '99 98\n', '999 550\n', '999 42\n', '999 25\n']


In [0]:
#interaction_graph = nx.read_edgelist("/out.petster-friendships-hamster-uniq", create_using = nx.Graph(), comments = "%")
interaction_graph = nx.read_edgelist("out.petster-friendships-hamster-uniq", create_using = nx.Graph(), comments = "%")

In [0]:
connected_subgraph=max(nx.connected_component_subgraphs(interaction_graph), key=len)


In [0]:
nx.info(connected_subgraph)

'Name: \nType: Graph\nNumber of nodes: 1788\nNumber of edges: 12476\nAverage degree:  13.9553'

## 3. Input

In [0]:
#define input
hidden_network=connected_subgraph
alpha1=0.1
alpha2=0.3
error_measure_aer=partial(add_edges_random, alpha=alpha1) 
error_measure_reu=partial(remove_edges_uniform, alpha=alpha1)
error_measure_repd=partial(remove_edges_proportional_degree, alpha=alpha1)
error_measure_rnu=partial(remove_nodes_uniform, alpha=alpha1)
centralityd=nk_wrapper(nk.centrality.DegreeCentrality, True)
centralityb=nk_wrapper(nk.centrality.Betweenness)
centralityc=nk_wrapper(nk.centrality.Closeness, True, True)
#centralitye=nx.eigenvector_centrality
centralitye=nk_wrapper(nk.centrality.EigenvectorCentrality)
centralityp=nk_wrapper(nk.centrality.PageRank)

degree_robustness_calculator = robustness_calculator_builder(centralityd)
betweenness_robustness_calculator = robustness_calculator_builder(centralityb)
centrality_robustness_calculator = robustness_calculator_builder(centralityc)
eigenvector_robustness_calculator = robustness_calculator_builder(centralitye)
pagerank_robustness_calculator = robustness_calculator_builder(centralityp)

## 4. Code

## add_edges_random 10 500

In [0]:
#interaction_graph = nx.read_edgelist("/out.petster-friendships-hamster-uniq", create_using = nx.Graph(), comments = "%")
interaction_graph = nx.read_edgelist("out.petster-friendships-hamster-uniq", create_using = nx.Graph(), comments = "%", nodetype=int)

In [0]:
true_robustness(hidden_network, error_measure_aer,degree_robustness_calculator)

robustness_true(mean=0.9219642159895846, sd=0.0021640089939342307, values=None)

In [0]:
estimate_robustness(error_measure_aer(hidden_network), error_measure_aer, degree_robustness_calculator,)

robustness_estimate(mean=0.922347391432886, sd=0.002156418969679542, values=None)

In [0]:
true_robustness(hidden_network, error_measure_aer,betweenness_robustness_calculator)

robustness_true(mean=0.6866478267563357, sd=0.007085650324672657, values=None)

In [0]:
estimate_robustness(error_measure_aer(hidden_network), error_measure_aer, betweenness_robustness_calculator,)

robustness_estimate(mean=0.7520403711242306, sd=0.0061366231395576535, values=None)

In [0]:
true_robustness(hidden_network, error_measure_aer,centrality_robustness_calculator)

robustness_true(mean=0.8891323888352974, sd=0.0031538638060063548, values=None)

In [0]:
estimate_robustness(error_measure_aer(hidden_network), error_measure_aer, centrality_robustness_calculator,)

robustness_estimate(mean=0.9011060474978956, sd=0.002704149918560122, values=None)

In [0]:
true_robustness(hidden_network, error_measure_aer,eigenvector_robustness_calculator)

robustness_true(mean=0.9305397848947347, sd=0.0026538530610510265, values=None)

In [0]:
estimate_robustness(error_measure_aer(hidden_network), error_measure_aer, eigenvector_robustness_calculator,)

robustness_estimate(mean=0.9295606301476871, sd=0.0025943596730706566, values=None)

In [0]:
true_robustness(hidden_network, error_measure_aer,pagerank_robustness_calculator)

robustness_true(mean=0.8371223288283208, sd=0.0033106730910927125, values=None)

In [0]:
estimate_robustness(error_measure_aer(hidden_network), error_measure_aer, pagerank_robustness_calculator,)

robustness_estimate(mean=0.86053422423693, sd=0.0027772244247031806, values=None)

##remove_edges_uniform 10 500 runs

In [0]:
true_robustness(hidden_network, error_measure_reu,degree_robustness_calculator)

robustness_true(mean=0.9612164131569961, sd=0.0013083567420279195, values=None)

In [0]:
estimate_robustness(error_measure_reu(hidden_network), error_measure_reu, degree_robustness_calculator,)

robustness_estimate(mean=0.9604088496102318, sd=0.001336044845537172, values=None)

In [0]:
true_robustness(hidden_network, error_measure_reu,betweenness_robustness_calculator)

robustness_true(mean=0.9175017428220412, sd=0.007063526431914543, values=None)

In [0]:
estimate_robustness(error_measure_reu(hidden_network), error_measure_reu, betweenness_robustness_calculator,)

robustness_estimate(mean=0.9157926797813332, sd=0.006965682030401637, values=None)

In [0]:
true_robustness(hidden_network, error_measure_reu,centrality_robustness_calculator)

robustness_true(mean=0.9320599340530468, sd=0.0036643203417553536, values=None)

In [0]:
estimate_robustness(error_measure_reu(hidden_network), error_measure_reu, centrality_robustness_calculator,)

robustness_estimate(mean=0.9303399856866509, sd=0.003705812490539374, values=None)

In [0]:
true_robustness(hidden_network, error_measure_reu,eigenvector_robustness_calculator)

robustness_true(mean=0.9360779309026731, sd=0.003153851268804703, values=None)

In [0]:
estimate_robustness(error_measure_reu(hidden_network), error_measure_reu, eigenvector_robustness_calculator,)

robustness_estimate(mean=0.9350873089263221, sd=0.003220138489252138, values=None)

In [0]:
true_robustness(hidden_network, error_measure_reu,pagerank_robustness_calculator)

robustness_true(mean=0.9262598228554376, sd=0.0030044252685534024, values=None)

In [0]:
estimate_robustness(error_measure_reu(hidden_network), error_measure_reu, pagerank_robustness_calculator,)

robustness_estimate(mean=0.9242494802453447, sd=0.0032885146916822126, values=None)

## remove_edges_proportional_degree 10 500 runs

In [0]:
true_robustness(hidden_network, error_measure_repd,degree_robustness_calculator)

robustness_true(mean=0.9725358081645531, sd=0.001029170544759112, values=None)

In [0]:
estimate_robustness(error_measure_repd(hidden_network), error_measure_repd, degree_robustness_calculator,)

robustness_estimate(mean=0.9714625849588805, sd=0.0010680765880117615, values=None)

In [0]:
true_robustness(hidden_network, error_measure_repd,betweenness_robustness_calculator)

robustness_true(mean=0.9395418865494446, sd=0.004324996080237535, values=None)

In [0]:
estimate_robustness(error_measure_repd(hidden_network), error_measure_repd, betweenness_robustness_calculator,)

robustness_estimate(mean=0.9381797954232258, sd=0.0042865182305489415, values=None)

In [0]:
true_robustness(hidden_network, error_measure_repd,centrality_robustness_calculator)

robustness_true(mean=0.914836345008822, sd=0.004337371428156348, values=None)

In [0]:
estimate_robustness(error_measure_repd(hidden_network), error_measure_repd, centrality_robustness_calculator,)

robustness_estimate(mean=0.9186183398550699, sd=0.003956159800167806, values=None)

In [0]:
true_robustness(hidden_network, error_measure_repd,eigenvector_robustness_calculator)

robustness_true(mean=0.9257509184069893, sd=0.0038664361680235434, values=None)

In [0]:
estimate_robustness(error_measure_repd(hidden_network), error_measure_repd, eigenvector_robustness_calculator,)

robustness_estimate(mean=0.9273106420381542, sd=0.0032557593745393284, values=None)

In [0]:
true_robustness(hidden_network, error_measure_repd,pagerank_robustness_calculator)

robustness_true(mean=0.9520777552481606, sd=0.0016684847712133434, values=None)

In [0]:
estimate_robustness(error_measure_repd(hidden_network), error_measure_repd, pagerank_robustness_calculator,)

robustness_estimate(mean=0.9501072258351904, sd=0.001967125492091114, values=None)

## remove_nodes_uniform 10 500  runs

In [0]:
true_robustness(hidden_network, error_measure_rnu,degree_robustness_calculator)

robustness_true(mean=0.9618512909656488, sd=0.004861204954265167, values=None)

In [0]:
estimate_robustness(error_measure_rnu(hidden_network), error_measure_rnu, degree_robustness_calculator,)

robustness_estimate(mean=0.9612869629131588, sd=0.005088233003214009, values=None)

In [0]:
true_robustness(hidden_network, error_measure_rnu,betweenness_robustness_calculator)

robustness_true(mean=0.9245750300637239, sd=0.010977955152898363, values=None)

In [0]:
estimate_robustness(error_measure_rnu(hidden_network), error_measure_rnu, betweenness_robustness_calculator,)

robustness_estimate(mean=0.9268256930961668, sd=0.01151611881167464, values=None)

In [0]:
true_robustness(hidden_network, error_measure_rnu,centrality_robustness_calculator)

robustness_true(mean=0.9330570633241395, sd=0.024657943065657183, values=None)

In [0]:
estimate_robustness(error_measure_rnu(hidden_network), error_measure_rnu, centrality_robustness_calculator,)

robustness_estimate(mean=0.9347580266484542, sd=0.025238199408695358, values=None)

In [0]:
true_robustness(hidden_network, error_measure_rnu,eigenvector_robustness_calculator)

robustness_true(mean=0.9324316708593833, sd=0.019063511534881873, values=None)

In [0]:
estimate_robustness(error_measure_rnu(hidden_network), error_measure_rnu, eigenvector_robustness_calculator,)

robustness_estimate(mean=0.934617213826352, sd=0.018324031040696947, values=None)

In [0]:
true_robustness(hidden_network, error_measure_rnu,pagerank_robustness_calculator)

robustness_true(mean=0.9292232154523516, sd=0.0059903642000597125, values=None)

In [0]:
estimate_robustness(error_measure_rnu(hidden_network), error_measure_rnu, pagerank_robustness_calculator,)

robustness_estimate(mean=0.93083342555535, sd=0.0065956032061427645, values=None)

In [0]:
alpha1=alpha2
print(alpha1)

0.3


## add_edges_random 30 500 runs


In [0]:
#interaction_graph = nx.read_edgelist("/out.petster-friendships-hamster-uniq", create_using = nx.Graph(), comments = "%")
interaction_graph = nx.read_edgelist("out.petster-friendships-hamster-uniq", create_using = nx.Graph(), comments = "%", nodetype=int)

In [0]:
true_robustness(hidden_network, error_measure_aer,degree_robustness_calculator)

robustness_true(mean=0.8531643592188435, sd=0.0034353003304243997, values=None)

In [0]:
estimate_robustness(error_measure_aer(hidden_network), error_measure_aer, degree_robustness_calculator,)

robustness_estimate(mean=0.8549545054576952, sd=0.003486673403026067, values=None)

In [0]:
true_robustness(hidden_network, error_measure_aer,betweenness_robustness_calculator)

robustness_true(mean=0.6291256582593223, sd=0.006523427888039022, values=None)

In [0]:
estimate_robustness(error_measure_aer(hidden_network), error_measure_aer, betweenness_robustness_calculator,)

robustness_estimate(mean=0.7307451786623244, sd=0.005241998478065331, values=None)

In [0]:
true_robustness(hidden_network, error_measure_aer,centrality_robustness_calculator)

robustness_true(mean=0.8081952330191543, sd=0.0038362965345421617, values=None)

In [0]:
estimate_robustness(error_measure_aer(hidden_network), error_measure_aer, centrality_robustness_calculator,)

robustness_estimate(mean=0.8342705200748955, sd=0.0033680979386678587, values=None)

In [0]:
true_robustness(hidden_network, error_measure_aer,eigenvector_robustness_calculator)

robustness_true(mean=0.867313244909482, sd=0.0033691335124201843, values=None)

In [0]:
estimate_robustness(error_measure_aer(hidden_network), error_measure_aer, eigenvector_robustness_calculator,)

robustness_estimate(mean=0.8677862393989265, sd=0.0032906062608151674, values=None)

In [0]:
true_robustness(hidden_network, error_measure_aer,pagerank_robustness_calculator)

robustness_true(mean=0.7626136307677321, sd=0.004480611625854604, values=None)

In [0]:
estimate_robustness(error_measure_aer(hidden_network), error_measure_aer, pagerank_robustness_calculator,)

robustness_estimate(mean=0.8022075879634519, sd=0.004001029652518447, values=None)

##remove_edges_uniform 30 500

In [0]:
true_robustness(hidden_network, error_measure_reu,degree_robustness_calculator)

robustness_true(mean=0.9005783182540931, sd=0.002453714968622882, values=None)

In [0]:
#2
true_robustness(hidden_network, error_measure_reu,degree_robustness_calculator)

robustness_true(mean=0.900654066090216, sd=0.002449839600132441, values=None)

In [0]:
estimate_robustness(error_measure_reu(hidden_network), error_measure_reu, degree_robustness_calculator,)

robustness_estimate(mean=0.8916232013963136, sd=0.002767840863365497, values=None)

In [0]:
true_robustness(hidden_network, error_measure_reu,betweenness_robustness_calculator)

robustness_true(mean=0.8119379839992654, sd=0.007822577582761796, values=None)

In [0]:
estimate_robustness(error_measure_reu(hidden_network), error_measure_reu, betweenness_robustness_calculator,)

robustness_estimate(mean=0.8041100647693852, sd=0.00758827920140934, values=None)

In [0]:
true_robustness(hidden_network, error_measure_reu,centrality_robustness_calculator)

robustness_true(mean=0.8311663213539261, sd=0.004932363118623273, values=None)

In [0]:
estimate_robustness(error_measure_reu(hidden_network), error_measure_reu, centrality_robustness_calculator,)

robustness_estimate(mean=0.8239038741643954, sd=0.004820895698923616, values=None)

In [0]:
true_robustness(hidden_network, error_measure_reu,eigenvector_robustness_calculator)

robustness_true(mean=0.8502252828071785, sd=0.004024945559188083, values=None)

In [0]:
estimate_robustness(error_measure_reu(hidden_network), error_measure_reu, eigenvector_robustness_calculator,)

robustness_estimate(mean=0.8360500415759239, sd=0.004735465847592928, values=None)

In [0]:
true_robustness(hidden_network, error_measure_reu,pagerank_robustness_calculator)

robustness_true(mean=0.840335988808296, sd=0.0041709787762109945, values=None)

In [0]:
estimate_robustness(error_measure_reu(hidden_network), error_measure_reu, pagerank_robustness_calculator,)

robustness_estimate(mean=0.8246300793946962, sd=0.004562223391188178, values=None)

## remove_edges_proportional_degree 30 500 runs


In [0]:
true_robustness(hidden_network, error_measure_repd,degree_robustness_calculator)

robustness_true(mean=0.9198434391081556, sd=0.0020460634580426336, values=None)

In [0]:
estimate_robustness(error_measure_repd(hidden_network), error_measure_repd, degree_robustness_calculator,)

robustness_estimate(mean=0.9089310872572858, sd=0.002361874577655201, values=None)

In [0]:
true_robustness(hidden_network, error_measure_repd,betweenness_robustness_calculator)

robustness_true(mean=0.8630177129499387, sd=0.005424805315856344, values=None)

In [0]:
estimate_robustness(error_measure_repd(hidden_network), error_measure_repd, betweenness_robustness_calculator,)

robustness_estimate(mean=0.8448354194780209, sd=0.006597280513660184, values=None)

In [0]:
true_robustness(hidden_network, error_measure_repd,centrality_robustness_calculator)

robustness_true(mean=0.7910096429575388, sd=0.004965659145106076, values=None)

In [0]:
estimate_robustness(error_measure_repd(hidden_network), error_measure_repd, centrality_robustness_calculator,)

robustness_estimate(mean=0.8045900492761235, sd=0.004545708325106268, values=None)

In [0]:
true_robustness(hidden_network, error_measure_repd,eigenvector_robustness_calculator)

robustness_true(mean=0.8138515621376744, sd=0.00562539644646531, values=None)

In [0]:
estimate_robustness(error_measure_repd(hidden_network), error_measure_repd, eigenvector_robustness_calculator,)

robustness_estimate(mean=0.8187595843771505, sd=0.005519623754446039, values=None)

In [0]:
true_robustness(hidden_network, error_measure_repd,pagerank_robustness_calculator)

robustness_true(mean=0.8916874800923593, sd=0.0025367869980571917, values=None)

In [0]:
estimate_robustness(error_measure_repd(hidden_network), error_measure_repd, pagerank_robustness_calculator,)

robustness_estimate(mean=0.8727795106416952, sd=0.0028380910414082363, values=None)

## remove_nodes_uniform 30 500 runs

In [0]:
true_robustness(hidden_network, error_measure_rnu,degree_robustness_calculator)

robustness_true(mean=0.9015503721912091, sd=0.009144234179107032, values=None)

In [0]:
estimate_robustness(error_measure_rnu(hidden_network), error_measure_rnu, degree_robustness_calculator,)

robustness_estimate(mean=0.8953062434787139, sd=0.010613946800134794, values=None)

In [0]:
true_robustness(hidden_network, error_measure_rnu,betweenness_robustness_calculator)

robustness_true(mean=0.8227528716008197, sd=0.015028984887349424, values=None)

In [0]:
estimate_robustness(error_measure_rnu(hidden_network), error_measure_rnu, betweenness_robustness_calculator,)

robustness_estimate(mean=0.8263853422277084, sd=0.015905608279603557, values=None)

In [0]:
true_robustness(hidden_network, error_measure_rnu,centrality_robustness_calculator)

robustness_true(mean=0.8387538662304656, sd=0.03296738334513333, values=None)

In [0]:
estimate_robustness(error_measure_rnu(hidden_network), error_measure_rnu, centrality_robustness_calculator,)

robustness_estimate(mean=0.8197226871866377, sd=0.04461114083651261, values=None)

In [0]:
true_robustness(hidden_network, error_measure_rnu,eigenvector_robustness_calculator)

robustness_true(mean=0.8467215741514384, sd=0.025134839394401802, values=None)

In [0]:
estimate_robustness(error_measure_rnu(hidden_network), error_measure_rnu, eigenvector_robustness_calculator,)

robustness_estimate(mean=0.8415432478872881, sd=0.028662251580990047, values=None)

In [0]:
true_robustness(hidden_network, error_measure_rnu,pagerank_robustness_calculator)

robustness_true(mean=0.8468938720787517, sd=0.009041562931487846, values=None)

In [0]:
estimate_robustness(error_measure_rnu(hidden_network), error_measure_rnu, pagerank_robustness_calculator,)

robustness_estimate(mean=0.8249760630595356, sd=0.013025849269846259, values=None)